# Introduction

Visual search is the common task of looking for something in a cluttered visual environment. The searched item is usually called the target, other non-target items are
called distractors. Many visual scenes contain more information than we can fully process all at once. This is the moment when the visual attention is essential. It is used to control the selection of the recognized objects. This selection could consist of objects, small groups of objects or even an array of locations. The selection periodicaly changes and usually consists of four to eight objects. 

The so called overt search refers to a series of eye movements around the scene made to bring the whole difficult-to-resolve item onto the fovea. On the other hand, if it is possible to identify the object during fixation the attention on a single point, the so called covert search happens (attentional shifts during a single fixation). Under dynamic real world conditions a new point of fixation is selected three to four times per second. However, under laboratory conditions, many search can be performed entirely with covert attention. 

The visual search experiment is a psychlological experiment used to analyze and study how the human visual search reacts on the specific stimuli in the visual scene. The experiment typicaly involves presenting a visual stimuli to a participant (picture, computer screen etc.). The stimuli typicaly contains number of objects. The participants are then asked to search for a specic target among the distractors (such as pink front slash as in our example). 

The researchers typicaly studied the effect of set size, target presence and homogeneity of the feature set on the accuracy and speed of the response.

# Experiment Description

During the experiment we studied how the reaction time and accuracy of the responses of the respondents differ under different visual stimuli. 

# TODO: popis experimentu


We recruited $N=3$ participants for the study. All the participants consisted of university male students in age range 22-23 years. The participants were seated in from of the notebook with 15.6 inch display with resolution 1920x1080 in a calm room. Then they were instructed to make themselves comfortable and turn off the notifications of their mobile phones. Afterwards, they were instructed about the experiment and performed the experiment. None of the participants did not need train trials or break. However, the participants were not fully concentrated during the first trial, so we decided to remove first 10 trials from the dataset.